In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import requests
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
we_go = pd.read_csv('../data/partner_project_data.csv')

In [3]:
new_frame = we_go[['DATE','ADHERENCE','HDWY_DEV','LATITUDE','LONGITUDE','OPERATOR','SCHEDULED_TIME',
       'ACTUAL_ARRIVAL_TIME', 'ACTUAL_DEPARTURE_TIME']]

In [4]:
new_frame_route22 = new_frame[new_frame['OPERATOR'].isin([1743, 2630, 3015, 1762, 2362])]

In [5]:
area_center = (36.14883, -86.73239)

In [6]:
i = 0
for row_index, row_values in new_frame_route22.iterrows():
    if i <3: 
        print('index is', row_index)
        print('values are:')
        print(' ')
        print(row_values)
        print('------------------------- ')
        i+=1

index is 6086
values are:
 
DATE                              2023-08-01
ADHERENCE                          -1.383333
HDWY_DEV                            1.016666
LATITUDE                            36.30047
LONGITUDE                          -86.69561
OPERATOR                                1743
SCHEDULED_TIME           2023-08-01 11:03:00
ACTUAL_ARRIVAL_TIME      2023-08-01 10:51:57
ACTUAL_DEPARTURE_TIME    2023-08-01 11:04:23
Name: 6086, dtype: object
------------------------- 
index is 6087
values are:
 
DATE                              2023-08-01
ADHERENCE                          -0.033333
HDWY_DEV                               -0.65
LATITUDE                           36.257412
LONGITUDE                         -86.714068
OPERATOR                                1743
SCHEDULED_TIME           2023-08-01 11:15:00
ACTUAL_ARRIVAL_TIME      2023-08-01 11:15:02
ACTUAL_DEPARTURE_TIME    2023-08-01 11:15:02
Name: 6087, dtype: object
------------------------- 
index is 6088
values are:
 


#draw our zip code area: 37207
publicartmap = folium.Map(location =  area_center, zoom_start = 12)

#folium.GeoJson(polygonpublicart).add_to(publicartmap)

#publicartzipurl = 'https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/tn_tennessee_zip_codes_geo.min.json'
#publicartzip = requests.get(publicartzipurl).json()
#folium.GeoJson(publicartzip, name='ZipCode').add_to(publicartmap)
#iterate through stops_in_37207 to create locations and markers 
#for each bus stop
#remember for Folium locations, lat is listed first!!

for row_index, row_values in new_frame_route22.iterrows():
    loc = [row_values['LATITUDE'], row_values['LONGITUDE']]
    pop = f"Scheduled_Time: {row_values['SCHEDULED_TIME']}<br>Arrival_Time:{row_values['ACTUAL_ARRIVAL_TIME']}<br>Departure_Time:{row_values['ACTUAL_DEPARTURE_TIME']}<br>Adherence:{row_values['ADHERENCE']}"
    icon=folium.Icon(color="orange",icon="bus", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop, icon = icon) 

    marker.add_to(publicartmap)
#map_37207.save('../maps/map37207.html')

#display our map
publicartmap

publicartzipurl = 'https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/tn_tennessee_zip_codes_geo.min.json'
publicartzip = requests.get(publicartzipurl).json()

# Create map with specified size
publicartmap = folium.Map(location=area_center, zoom_start=12, width='80%', height='80%')


folium.GeoJson(publicartzip, name='ZipCode').add_to(publicartmap)

# Iterate through stops and create markers
for row_index, row_values in new_frame_route22.iterrows():
    loc = [row_values['LATITUDE'], row_values['LONGITUDE']]
    pop = f"Scheduled_Time: {row_values['SCHEDULED_TIME']}<br>Arrival_Time:{row_values['ACTUAL_ARRIVAL_TIME']}<br>Departure_Time:{row_values['ACTUAL_DEPARTURE_TIME']}<br>Adherence:{row_values['ADHERENCE']}"
    icon = folium.Icon(color="orange", icon="bus", prefix='fa')
    
    marker = folium.Marker(location=loc, popup=pop, icon=icon)
    marker.add_to(publicartmap)

# Save and display the map
#publicartmap.save('../maps/map37207.html')
publicartmap

In [8]:
busmap = folium.Map(location=[36.3, -86.7], zoom_start=12, width='80%', height='80%')

# Group by operator
grouped = new_frame_route22.groupby('OPERATOR')

# Define colors for different operators
colors = ['blue', 'green', 'red', 'purple', 'orange']

# Iterate through each group
for i, (operator, group) in enumerate(grouped):
    points = []
    for row_index, row_values in group.iterrows():
        loc = [row_values['LATITUDE'], row_values['LONGITUDE']]
        points.append(loc)  # Add the location to the points list
        pop = f"Scheduled_Time: {row_values['SCHEDULED_TIME']}<br>Arrival_Time:{row_values['ACTUAL_ARRIVAL_TIME']}<br>Departure_Time:{row_values['ACTUAL_DEPARTURE_TIME']}<br>Adherence:{row_values['ADHERENCE']}"
        icon = folium.Icon(color="orange", icon="bus", prefix='fa')
        
        marker = folium.Marker(location=loc, popup=pop, icon=icon)
        marker.add_to(busmap)
    
    # Add the PolyLine to trace the route for each operator
    folium.PolyLine(points, color=colors[i % len(colors)], weight=2.5, opacity=1).add_to(busmap)

# Display the map
busmap
busmap.save('busmap.html')